# Basic Kubeflow Pipeline with Control Flow Logic

Demonstrate how to build Python components and a simple pipeline and a simple if statement.

1. Creqte the components. There is a flip_coin component that determine if we add or multiply.
2. Define the pipeline. Note the if logic that branches the pipeline based on the coin flip.

In [ ]:
! pip install --upgrade --quiet google-cloud-aiplatform kfp google-cloud-storage

## Set the required libraries and variables

In [ ]:
import os
import kfp
from kfp import compiler
from kfp.dsl import component, pipeline, If
from google.cloud import aiplatform

# Change these values to match your project setup
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

REGION = "us-central1" # @param {type:"string"}

# An existing bucket to store pipeline artifacts.
BUCKET_NAME = "basic-kf-pipelines-mlops-dar" # @param {type:"string"}

# Set default values for aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=f"gs://{BUCKET_NAME}")


# Create the Components

In [ ]:
@component(
    packages_to_install=["pandas"],
    base_image="python:3.9"
)
def say_hello(name: str) -> str:
    """
    A simple component that returns a 'Hello <name>!' string.
    """
    message = f"Hello {name}!"
    print(message)
    return message

@component(
    packages_to_install=[],
    base_image="python:3.9"
)
def add_numbers(x: float, y: float) -> float:
    """
    Adds two numbers and returns the result.
    """
    result = x + y
    print(f"Adding {x} + {y} = {result}")
    return result

@component(
    packages_to_install=[],
    base_image="python:3.9"
)
def multiply_numbers(x: float, y: float) -> float:
    """
    Multiplies two numbers and returns the result.
    """
    result = x * y
    print(f"Multiplying {x} * {y} = {result}")
    return result

@component(
    packages_to_install=[],
    base_image="python:3.9"
)
def flip_coin() -> str:
    """
    Returns 'heads' or 'tails' randomly.
    """
    import random
    return "heads" if random.random() < 0.5 else "tails"

@component(
    packages_to_install=[],
    base_image="python:3.9"
)
def compose_message(name: str, x: float, y: float, operation: str, result: float) -> str:
    """
    Composes a message that includes the name, the two numbers,
    the operation ('plus' or 'times'), and the result.
    """
    message = f"Thanks, {name}. {x} {operation} {y} = {result}."
    print(message)
    return message


## Build the Pipeline

Note the If logic

In [ ]:
@pipeline(
    name="simple-demo-pipeline-with-flip",
    description="A pipeline that prints a greeting, flips a coin, then either multiplies or adds two numbers, and composes a final message."
)
def my_simple_pipeline_with_flip(
    name_value: str = "Joe",
    x_value: float = 3.0,
    y_value: float = 4.0
):
    # Step 1: Say hello
    hello_task = say_hello(name=name_value)

    # Step 2: Flip coin
    coin_task = flip_coin()

    # Step 3: If heads => multiply
    with If(coin_task.output == "heads"):
        multiply_task = multiply_numbers(x=x_value, y=y_value)
        compose_message(
            name=name_value,
            x=x_value,
            y=y_value,
            operation="times",
            result=multiply_task.output
        )

    # Step 4: If tails => add
    with If(coin_task.output == "tails"):
        add_task = add_numbers(x=x_value, y=y_value)
        compose_message(
            name=name_value,
            x=x_value,
            y=y_value,
            operation="plus",
            result=add_task.output
        )



## Compile the Pipeline to a JSON file

In [ ]:
from kfp.v2 import compiler

pipeline_filename = "my_simple_pipeline_job_wit_flip.json"
compiler.Compiler().compile(
    pipeline_func=my_simple_pipeline_with_flip,
    package_path=pipeline_filename
)

print(f"Pipeline compiled to {pipeline_filename}")


Pipeline compiled to my_simple_pipeline_job_wit_flip.json


## Submit the job to Vertex AI Pipelines

In [ ]:
from google.cloud.aiplatform import PipelineJob

# Create a PipelineJob and submit
job = PipelineJob(
    display_name="my-simple-demo-pipeline-job",
    template_path=pipeline_filename,  # The JSON artifact produced by compilation
    pipeline_root=f"gs://{BUCKET_NAME}/pipeline_root",  # Where Vertex AI will store pipeline artifacts
    parameter_values={
        "name_value": "Steve", # override default
        "x_value": 27.4,  # override default
        "y_value": 53.22,  # override default
    },
    enable_caching=False,  # Need to turn caching off or the coin flip is the same everytime
)

# Note: If caching is true, but the parameter change the task is re-run.
# If caching is true, but the parameters don't change, tasks are not re-run.

# Runs the job and waits for it to finish
# job.run()

# Submits the job to Vertex AI Pipelines and completes immediately
job.submit()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/117114503109/locations/us-central1/pipelineJobs/simple-demo-pipeline-with-flip-20250208143834
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/117114503109/locations/us-central1/pipelineJobs/simple-demo-pipeline-with-flip-20250208143834')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/simple-demo-pipeline-with-flip-20250208143834?project=117114503109
